In [6]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal
import pywt


data_folder = './data/'
wav_folder_rx ='./samples_A_RX1_C0/'
wav_folder_deformed ='./samples_A_DEFORMED1_C0/'

#CSV filenames
deformed01 = 'A_DEFORMED1_C0'
deformed02 = 'A_DEFORMED1_C0'
rx01 = 'A_RX1_C0'

#WAV filenames
wav_rx = 'sample_A_RX1_C00'

df = pd.read_csv(data_folder + deformed02 + '.csv', header = None)

In [ ]:
data_folder = './data/'
wav_folder_rx ='./samples_A_RX1_C0/'
wav_folder_deformed ='./samples_A_DEFORMED1_C0/'

#CSV filenames
deformed01 = 'A_DEFORMED1_C0'
deformed02 = 'A_DEFORMED1_C0'
rx01 = 'A_RX1_C0'

#WAV filenames
wav_rx = 'sample_A_RX1_C00'

df = pd.read_csv(data_folder + deformed02 + '.csv', header = None)

In [3]:

#initialisation of clock variables
clock = 3048780.48780488
delay_in_samples = 304878
g_T = 1 / clock
g_N = len(df)

print("the frequency of the signal is", clock * 1e-6, "MHz")
print("the total duration of the signal is", g_N / clock, "seconds")
print("number of samples :", len(df))


the frequency of the signal is 3.04878048780488 MHz
the total duration of the signal is 10.495999999999993 seconds
number of samples : 32000000


In [4]:
#remove the last points so that the array length is divisible by the number of points per sample 
#in order to have equal lengths sample
def clean_data(df):
    epsilon =  1e-1
    #remove the silence at the end: 
    for i in range(len(df) - 1, 0, -1):
        if(df[0][i] > epsilon):
            print("deleted", (len(df) - i) / clock, "samples from the end because we assume it is silence from", len(df) / clock, "samples")
            return df[:i]

def split_csv(filename, T):
    df = pd.read_csv(data_folder + filename + '.csv', header = None)
    df = clean_data(df)
    df.rename(columns = {0:'amplitude'}, inplace = True)
    point_per_sample = int(160e-3 * clock)
    
    nb_of_samples = int(len(df)/point_per_sample)
    data = df[:nb_of_samples * point_per_sample]
    return np.array_split(data, nb_of_samples)

data_deformed = split_csv(deformed01, g_T)
#data_rx = split_csv(rx01, g_T)


deleted 1.3556525359999991 samples from the end because we assume it is silence from 10.495999999999993 samples


In [8]:


#Wavelets
t = np.linspace(-1, 1, 200, endpoint=False)
flat_list = []
widths = np.arange(1, 31)
for sublist in data_deformed:
    for item in sublist['amplitude']:
        flat_list.append(item)

coef, freqs = pywt.cwt(flat_list,np.arange(1,129), 'morl' )
plt.matshow(coef)
# cwt has shape(30, 128795) 30 being widths
cwt = signal.cwt(flat_list, signal.morlet, widths)

In [ ]:
#standardize data
normalized_cwt=(cwt-cwt.mean())/cwt.std()
print(normalized_cwt)



[[ 1.89715206e-03  8.17831480e-04 -2.01714086e-03 ...  3.49778435e-01
   3.63097780e+00  9.40086257e+00]
 [-5.86234753e-03  5.95608922e-03  1.70033451e-02 ...  9.84106609e+00
   1.24308497e+01  9.12124329e+00]
 [ 5.87897810e-04  1.56878155e-02  2.93046863e-02 ...  1.50447341e+01
   1.32753235e+01  7.85236807e+00]
 ...
 [-2.83781863e-02 -2.76508467e-02 -2.68712364e-02 ...  8.04028751e+00
   5.38709406e+00  2.71342071e+00]
 [-2.92518314e-02 -2.84240012e-02 -2.75411792e-02 ...  7.90592470e+00
   5.29678984e+00  2.66888301e+00]
 [-2.99748840e-02 -2.90545801e-02 -2.80775123e-02 ...  7.77812810e+00
   5.21100283e+00  2.62661995e+00]]


In [ ]:
#PCA
from sklearn.decomposition import PCA
pca = PCA(len(normalized_cwt))
pca.fit(normalized_cwt)
train_img = pca.transform(normalized_cwt)

#shape is 30x30
print(train_img)

[[-3.66691606e+03  1.60707417e+03  8.62882019e+02  5.03116238e+02
   3.05703911e+02  1.70466121e+02 -7.28256539e+01 -2.25276098e+01
   5.52238470e+00  1.19980041e+00 -2.38624269e-01  4.12777477e-02
   6.52545543e-03 -9.46985738e-04  1.24474778e-04  1.55030249e-05
  -2.37972812e-06 -5.88896672e-07  3.93805713e-07  2.57809819e-07
  -1.61485837e-07  9.44756915e-08 -7.04194296e-08 -7.22402718e-08
  -2.97015576e-08 -1.99434356e-09  4.88774346e-09 -6.69350275e-09
   3.74156271e-09 -1.99179415e-10]
 [-3.68110730e+03  1.59565883e+03  7.92565600e+02  3.38921429e+02
   5.56362007e+01 -9.62823356e+01  1.07277197e+02  5.64184958e+01
  -1.96139318e+01 -5.40268191e+00  1.25711779e+00 -2.47675347e-01
  -4.26505371e-02  6.66095603e-03 -9.40812230e-04 -1.25470391e-04
   1.96598006e-05  5.09067553e-06 -3.45273513e-06 -2.05892873e-06
   1.54365754e-06 -7.23056417e-07  7.05198767e-07  5.45870990e-07
   1.49003430e-07  1.23404847e-07  2.29488690e-08  2.48381488e-08
  -1.39988611e-08 -1.59282439e-10]
 [-3.6

In [ ]:
#https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60


30


In [ ]:
import matplotlib.pyplot as plt
import pywt
coef, freqs = pywt.cwt(flat_list,np.arange(1,31), 'morl' )
plt.matshow(coef)
plt.show()

NameError: name 'flat_list' is not defined